In [4]:
from gql import gql, Client

import psycopg2
import pandas as pd

import json
from tqdm import tqdm

## DATA

In [2]:
# Establish a connection to the database
conn = psycopg2.connect(
    dbname="bali5",
    user=user1,
    password=password1,
    host="pg.squid.subsquid.io",
    port="5432"
)

In [ ]:
# Create a cursor from the connection
cur = conn.cursor()

# Query the columns of a specific table
cur.execute("""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = 'era_staker'
""")

# Fetch all rows from the cursor
columns = cur.fetchall()

# Print the columns
for column in columns:
    print(column)

In [2]:
# df = pd.read_pickle('nominator_data_full.pkl')

In [9]:
df.era_id.nunique()

1125

In [86]:
df.validatorId.nunique()

1185

In [87]:
df.nominatorId.nunique()

80709

In [89]:
len(final_df)

19157515

In [97]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact, IntSlider


# Create a pivot table
pivot_table = df.pivot_table(values='totalBonded', index='nominatorId', columns='validatorId')

@interact
def plot_heatmap(topk=IntSlider(min=1, max=50, value=10)):
    # Select top k nominators and validators
    top_nominators = pivot_table.sum(axis=1).nlargest(topk).index
    top_validators = pivot_table.sum(axis=0).nlargest(topk).index

    # Select corresponding rows and columns from pivot table
    subset = pivot_table.loc[top_nominators, top_validators]

    # Plot heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(subset, cmap='viridis')
    plt.title(f"Heatmap for Top {topk} Validators and Nominators")
    plt.xlabel("Validators")
    plt.ylabel("Nominators")
    plt.show()

interactive(children=(IntSlider(value=10, description='topk', max=50, min=1), Output()), _dom_classes=('widget…

In [109]:
df_agg = final_df

In [ ]:
import plotly.graph_objects as go

# Create a list of labels (nominatorId and validatorId)
labels = list(df_agg['nominatorId'].unique()) + list(df_agg['validatorId'].unique())

# Create a list of colors for the labels, here for simplicity, nominators are colored blue and validators red
colors = ['blue'] * len(df_agg['nominatorId'].unique()) + ['red'] * len(df_agg['validatorId'].unique())

# Create a dictionary mapping for the labels
label_dict = {label: idx for idx, label in enumerate(labels)}

# Prepare source, target and value lists
source = df_agg['nominatorId'].map(label_dict)
target = df_agg['validatorId'].map(label_dict)
value = df_agg['amount']

# Create a Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color='black', width=0.5),
        label=labels,
        color=colors
    ),
    link=dict(
        source=source,  # indices correspond to labels
        target=target,  # indices correspond to labels
        value=value
    )
)])

fig.update_layout(title_text='Flow of Amounts from Nominators to Validators', font_size=10)
fig.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import imageio

# Preprocess the data: normalize 'amount' for each era
df['normalized_amount'] = df.groupby('era_id')['amount'].apply(lambda x: x / x.max())

# Get the list of eras
eras = df['era_id'].unique()

# Create a directory for storing images if not exists
import os
if not os.path.exists('images'):
    os.makedirs('images')

# Create and save a heatmap for each era
for era in eras:
    # Get the data for the era
    df_era = df[df['era_id'] == era]

    # Create a pivot table
    pivot_table = df_era.pivot_table(values='normalized_amount', index='nominatorId', columns='validatorId', fill_value=0)

    # Plot heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(pivot_table, cmap='viridis')
    plt.title(f"Era {era}")

    # Save the heatmap as an image
    plt.savefig(f'images/heatmap_{era}.png')
    plt.close()

# Create a video from images
images = [imageio.imread(f'images/heatmap_{era}.png') for era in eras]
imageio.mimsave('heatmap_video.gif', images, fps=10)


In [5]:
df = pd.read_pickle('reward_percentage_with_amount.pkl')

In [ ]:
df

In [ ]:
import os
import seaborn as sns
import imageio
import matplotlib.pyplot as plt

# Create a copy of the DataFrame and sort it
sorted_df = df.sort_values(['nominator_id', 'validator_id', 'era_id'])

# Create a new column 'previous_era' with the era of the previous nomination for each nominator and validator
sorted_df['previous_era'] = sorted_df.groupby(['nominator_id', 'validator_id'])['era_id'].shift()

# Create a column to identify new nominators or those returning after an absence of 10 eras
sorted_df['new_nominator'] = ((sorted_df['era_id'] - sorted_df['previous_era']) >= 10) | sorted_df['nominator_id'].ne(sorted_df['nominator_id'].shift())

# Create an aggregated dataframe to get the total number of new nominators for each validator in each era
agg_df = sorted_df.groupby(['era_id', 'validator_id', 'reward_percentage']).agg(
    num_nominators=('nominator_id', 'count'),
    new_nominators=('new_nominator', 'sum')
).reset_index()

# Calculate max and min values for axes
max_reward_percentage = agg_df['reward_percentage'].max()
max_num_nominators = agg_df['num_nominators'].max()
min_reward_percentage = max_reward_percentage / -100
min_num_nominators = max_num_nominators / -100

# Create a directory to store the images
if not os.path.exists('images'):
    os.makedirs('images')

# Loop over each era to create the plots
for era in agg_df['era_id'].unique():
    era_df = agg_df[agg_df['era_id'] == era]

    # Create a scatter plot with different colors for new/returning nominators
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=era_df, x='reward_percentage', y='num_nominators', size='new_nominators', hue='new_nominators', palette='viridis')
    plt.xlabel('Reward Percentage')
    plt.ylabel('Number of Nominators')
    plt.title(f'Effect of Reward Percentage on Validator Choice in Era {era}')
    plt.xlim(min_reward_percentage, max_reward_percentage)
    plt.ylim(min_num_nominators, max_num_nominators)
    plt.savefig(f'images/era_{era}.png')
    plt.close()

# Create the GIF as before
images = [imageio.imread(f'images/era_{era}.png') for era in agg_df['era_id'].unique()]
imageio.mimsave('reward_vs_nominators.gif', images, duration=20)

In [ ]:
import os
import seaborn as sns
import imageio
import matplotlib.pyplot as plt

# Create a copy of the DataFrame and sort it
sorted_df = df.sort_values(['nominator_id', 'validator_id', 'era_id'])

# Create a new column 'previous_era' with the era of the previous nomination for each nominator and validator
sorted_df['previous_era'] = sorted_df.groupby(['nominator_id', 'validator_id'])['era_id'].shift()

# Create a column to identify new nominators or those returning after an absence of 10 eras
sorted_df['new_nominator'] = ((sorted_df['era_id'] - sorted_df['previous_era']) >= 10) | sorted_df['nominator_id'].ne(sorted_df['nominator_id'].shift())

# Create an aggregated dataframe to get the total number of new nominators for each validator in each era
agg_df = sorted_df.groupby(['era_id', 'validator_id', 'reward_percentage']).agg(
    num_nominators=('nominator_id', 'count'),
    new_nominators=('new_nominator', 'sum')
).reset_index()

# Exclude groups where we do not have any nominators
agg_df = agg_df[agg_df['num_nominators'] > 0]

# Calculate max and min values for axes
max_reward_percentage = agg_df['reward_percentage'].max()
max_num_nominators = agg_df['num_nominators'].max()
min_reward_percentage = max_reward_percentage / -100
min_num_nominators = max_num_nominators / -100

# Create a directory to store the images
if not os.path.exists('images'):
    os.makedirs('images')

# Loop over each era to create the plots
for era in agg_df['era_id'].unique():
    era_df = agg_df[agg_df['era_id'] == era]

    # Create a scatter plot with different colors for new/returning nominators
    plt.figure(figsize=(10, 6))
    plot = sns.scatterplot(data=era_df, x='reward_percentage', y='num_nominators',
                           size='new_nominators', hue='new_nominators',
                           palette='viridis')

    plt.xlabel('Reward')
    plt.ylabel('Number of Nominators')
    plt.title(f'Effect of Reward on Validator Choice in Era {era}')
    plt.xlim(min_reward_percentage, max_reward_percentage)
    plt.ylim(min_num_nominators, max_num_nominators)

    # Fix the legend to start from 1
    handles, labels = plot.get_legend_handles_labels()
    labels[1] = '1'  # Change the label '0' to '1'
    plot.legend(handles=handles[1:], labels=labels[1:], title='New Nominators')

    plt.savefig(f'images/era_{era}.png')
    plt.close()



# Create the GIF as before
images = [imageio.imread(f'images/era_{era}.png') for era in agg_df['era_id'].unique()]
imageio.mimsave('reward_vs_nominators.gif', images, duration=50)


In [ ]:
images = [imageio.imread(f'images/era_{era}.png') for era in agg_df['era_id'].unique()]
imageio.mimsave('reward_vs_nominators.gif', images, duration=20)

In [ ]:
from matplotlib.animation import FuncAnimation
from PIL import Image

fig, ax = plt.subplots(figsize=(10, 6))

def update(era):
    ax.clear()
    era_df = agg_df[agg_df['era_id'] == era]
    plot = sns.scatterplot(data=era_df, x='reward_percentage', y='num_nominators',
                           size='new_nominators', hue='new_nominators',
                           palette='viridis', ax=ax)
    ax.set_xlabel('Reward')
    ax.set_ylabel('Number of Nominators')
    ax.set_title(f'Effect of Reward on Validator Choice in Era {era}')
    ax.set_xlim(min_reward_percentage, max_reward_percentage)
    ax.set_ylim(min_num_nominators, max_num_nominators)
    # Fix the legend to start from 1
    handles, labels = ax.get_legend_handles_labels()
    labels[1] = '1'  # Change the label '0' to '1'
    ax.legend(handles=handles[1:], labels=labels[1:], title='New Nominators')

ani = FuncAnimation(fig, update, frames=agg_df['era_id'].unique())

ani.save('reward_vs_nominators.mp4', writer='ffmpeg', fps=5)
plt.close(fig)


In [ ]:
import os
import seaborn as sns
import imageio
import matplotlib.pyplot as plt
import numpy as np
import io

# Create a copy of the DataFrame and sort it
sorted_df = df.sort_values(['nominator_id', 'validator_id', 'era_id'])

# Create a new column 'previous_era' with the era of the previous nomination for each nominator and validator
sorted_df['previous_era'] = sorted_df.groupby(['nominator_id', 'validator_id'])['era_id'].shift()

# Create a column to identify new nominators or those returning after an absence of 10 eras
sorted_df['new_nominator'] = ((sorted_df['era_id'] - sorted_df['previous_era']) >= 10) | sorted_df['nominator_id'].ne(sorted_df['nominator_id'].shift())

# Create an aggregated dataframe to get the total number of new nominators for each validator in each era
agg_df = sorted_df.groupby(['era_id', 'validator_id', 'reward_percentage']).agg(
    num_nominators=('nominator_id', 'count'),
    new_nominators=('new_nominator', 'sum')
).reset_index()

# Exclude groups where we do not have any nominators
agg_df = agg_df[agg_df['num_nominators'] > 0]

# Calculate max and min values for axes
max_reward_percentage = agg_df['reward_percentage'].max()
max_num_nominators = agg_df['num_nominators'].max()
min_reward_percentage = max_reward_percentage / -100
min_num_nominators = max_num_nominators / -100

# Create a directory to store the images
if not os.path.exists('images'):
    os.makedirs('images')

# Create a list to store image arrays
images = []

# Loop over each era to create the plots
for era in agg_df['era_id'].unique():
    era_df = agg_df[agg_df['era_id'] == era]

    # Create a scatter plot with different colors for new/returning nominators
    plt.figure(figsize=(10, 6))
    plot = sns.scatterplot(data=era_df, x='reward_percentage', y='num_nominators', size='new_nominators', hue='new_nominators', palette='viridis')
    plt.xlabel('Reward Percentage')
    plt.ylabel('Number of Nominators')
    plt.title(f'Effect of Reward Percentage on Validator Choice in Era {era}')
    plt.xlim(min_reward_percentage, max_reward_percentage)
    plt.ylim(min_num_nominators, max_num_nominators)

    # Fix the legend to start from 1
    handles, labels = plot.get_legend_handles_labels()
    plot.legend(handles=handles[1:], labels=np.arange(1, len(handles)), title='New Nominators')

    # Save the figure to a file-like object
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    image = imageio.imread(buf)
    images.append(image)
    plt.close()

# Save images as MP4
imageio.mimwrite('reward_vs_nominators.mp4', images, dutation=40)
